# 23_RPeaks_Inicial

Extração de traçado 1D e detecção de R-peaks para estimativa de FC.

In [ ]:
import numpy as np
from PIL import Image

from cv.grid_detect import estimate_grid_period_px
from cv.rpeaks_from_image import (
    detect_rpeaks_from_trace,
    estimate_px_per_sec,
    extract_trace_centerline,
    smooth_signal,
)
from cv.segmentation import find_content_bbox
from cv.segmentation_ext import segment_layout

img = Image.open('../samples/ecg_images/synthetic_12lead.png').convert('L')
arr = np.asarray(img); from PIL import Image as P
rgb = P.open('../samples/ecg_images/synthetic_12lead.png').convert('RGB')
bbox = find_content_bbox(arr)
seg = segment_layout(arr, '3x4', bbox=bbox)
lab2box = {d['lead']: d['bbox'] for d in seg}
x0,y0,x1,y1 = lab2box['II']
crop = arr[y0:y1,x0:x1]
trace = smooth_signal(extract_trace_centerline(crop), win=11)
pxmm = (estimate_grid_period_px(np.asarray(rgb)).get('px_small_x') or 10)
pxsec = estimate_px_per_sec(pxmm, 25.0)
res = detect_rpeaks_from_trace(trace, px_per_sec=pxsec)
res